In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import numpy as np


In [29]:
data="C://Users//USER//Desktop//data"

In [30]:
file_list = os.listdir(data)
soup_objects = []
attributes_list = [
    'لون السيارة', 'نوع الوقود', 'أصل السيارة', 'رخصة السيارة', 'نوع الجير',
    'الزجاج', 'قوة الماتور', 'عداد السيارة', 'عدد الركاب', 'وسيلة الدفع',
    'معروضة', 'أصحاب سابقون', 'إضافات'
]
attributes_data = {attr: [] for attr in attributes_list}


In [31]:
def get_price(soup):
    car_prices = soup.find_all('h5', class_='post-price')

    if car_prices:
        prices = []

        for price_element in car_prices:
            price = price_element.text.strip().split()[0]
            prices.append(price)

        return prices
    else:
        return None


In [32]:
def get_model(soup):
    driving_table = soup.find("table", class_="driving-table")

    if driving_table:
        car_model_tag = driving_table.find("h3")

        if car_model_tag:
            model = car_model_tag.text.strip()
        else:
            model = None
    else:
        model = None

    return model


In [33]:
def get_year(soup):
    driving_table = soup.find("table", class_="driving-table")

    if driving_table:
        car_year_tag = driving_table.find("h5")

        if car_year_tag:
            car_year_text = car_year_tag.text.strip()
            # Extract numeric part from the text
            numeric_part = ''.join(filter(str.isdigit, car_year_text))
            year = int(numeric_part) if numeric_part else None
        else:
            year = None
    else:
        year = None

    return year


In [34]:
def get_car_attributes(soup):
    car_data = {}

    # Find the table with class 'list_ads'
    table = soup.find('table', class_='list_ads')

    if table:
        # Find all rows in the table
        rows = table.find_all('tr', class_='list-row')

        # Initialize an empty dictionary to store attributes
        attributes = {}

        for row in rows:
            # Extract cells from each row
            cells = row.find_all('td')

            # Check if there are at least two cells
            if len(cells) == 2:
                # Extract attribute name and value
                attribute_name = cells[0].text.strip()
                attribute_value = cells[1].text.strip()

                # Store the attribute in the dictionary
                attributes[attribute_name] = attribute_value

        # Add the 'attributes' key to the car_data dictionary
        car_data['attributes'] = attributes

    return car_data


In [35]:
prices = []
models = []
years = []
attributes_data = {attr: [] for attr in attributes_list}

for file_name in file_list:
    file_path = os.path.join(data, file_name)

    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')

        price = get_price(soup)
        model = get_model(soup)
        year = get_year(soup)
        car_data = get_car_attributes(soup)

        if 'attributes' in car_data:
            attribute_values = car_data['attributes']
        else:
            attribute_values = {}

        prices.append(price if price else np.nan)
        models.append(model if model else np.nan)
        years.append(year if year else np.nan)

        for attr, value in attributes_data.items():
            attributes_data[attr].append(attribute_values.get(attr, np.nan))


In [39]:
data = {
    'Price': prices,
    'Model': models,
    'Year': years,
    **attributes_data
}

df = pd.DataFrame(data)
df.to_csv('carsData_saef.csv', index=False)
df.head()


,Price,Model,Year,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,إضافات
0,[100000],كيا اوبتيما,2014.0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,4+1,نقدا فقط,للبيع فقط,يد اولى,مُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\nفتحة...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[60000],كيا سورينتو,2007.0,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,7+1,إمكانية التقسيط,للبيع أو التبديل,2,مُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\nفتحة...
4,[43500],هونداي افانتي,2006.0,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,NaN,نقدا فقط,للبيع فقط,NaN,مُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\nجنطا...
